In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, clear_output
import time

In [2]:
# Function to count fingers
def count_fingers(contour, defects):
    count = 0
    for defect in defects:
        start, _, end, _ = defect[0]
        start_point = tuple(contour[start][0])
        end_point = tuple(contour[end][0])
        far_point = tuple(contour[defect[0][2]][0])

        # Use cosine rule to find the angle of the defect
        a = np.sqrt((end_point[0] - start_point[0])**2 + (end_point[1] - start_point[1])**2)
        b = np.sqrt((far_point[0] - start_point[0])**2 + (far_point[1] - start_point[1])**2)
        c = np.sqrt((end_point[0] - far_point[0])**2 + (end_point[1] - far_point[1])**2)

        angle = np.arccos((b**2 + c**2 - a**2) / (2*b*c))

        # If the angle is below a certain threshold, consider it a finger
        if angle <= np.pi/2:
            count += 1

    return count

In [3]:
# Function for hand detection and segmentation
def detect_and_segment_hand(frame):
    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for skin color in HSV
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Create a binary mask to segment the hand
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Apply morphological operations to enhance the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=4)
    mask = cv2.erode(mask, kernel, iterations=4)

    # Bitwise AND operation to extract the hand region
    segmented_hand = cv2.bitwise_and(frame, frame, mask=mask)

    return segmented_hand


In [4]:
# Function to process video stream and perform finger counting
def process_video():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        # Detect and segment the hand
        segmented_hand = detect_and_segment_hand(frame)

        # Find contours and convex hull
        contours, _ = cv2.findContours(cv2.cvtColor(segmented_hand, cv2.COLOR_BGR2GRAY),
                                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            max_contour = max(contours, key=cv2.contourArea)
            hull = cv2.convexHull(max_contour, returnPoints=False)
            defects = cv2.convexityDefects(max_contour, hull)

            # Count fingers and display the result
            finger_count = count_fingers(max_contour, defects)
            cv2.putText(frame, f"Fingers: {finger_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame using matplotlib to work in Jupyter Notebook
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

        # Clear previous output to update the displayed frame
        clear_output(wait=True)
        time.sleep(0.1)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
# Run the video processing function
process_video()

KeyboardInterrupt: 